In [1]:
# Загружаем бибилиотеки
import pandas as pd
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker
from dateutil.relativedelta import relativedelta

In [2]:
# Создаём подключение
conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=192.168.189.71;DATABASE=Dokuchaev;UID=sa;PWD=As12345^'
quoted = quote_plus(conn_string)
new_conn_string = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_conn_string)

In [3]:
%%time
sales = pd.read_sql('SELECT * FROM dbo.sales_tb', engine)

Wall time: 3.12 s


In [4]:
sales.head(5)

,Store.Id,name,year,month,orders,sales,discount,type,code
0,dc590a6f-a302-4883-af69-8d612e273130,Лесная 43_СТ_МСК,2020,1,175,27060.0,360.0,STORE,1221-12
1,277334f1-0d3c-4818-a934-81641a83c2bf,Новый Арбат 3_СТ_МСК,2020,1,379,55440.0,960.0,STORE,2125
2,c27d8f86-50b9-49be-9abe-7a9b8ebc8fe3,Архив Метрополис 1_ТЦ_МСК,2018,8,638,86980.0,1260.0,STORE,12
3,bc5d3170-e307-468a-9437-0609f3bcf91d,Павелецкий_ТХ_МСК,2018,8,728,92000.0,1980.0,STORE,110
4,681f37cd-1166-4ead-a7b9-6c29fd4237b4,Свиблово_ТРЦ_МСК,2019,9,386,51000.0,1320.0,STORE,97


In [5]:
sales[['Store.Id','name','year', 'month', 'sales']]

,Store.Id,name,year,month,sales
0,dc590a6f-a302-4883-af69-8d612e273130,Лесная 43_СТ_МСК,2020,1,27060.0
1,277334f1-0d3c-4818-a934-81641a83c2bf,Новый Арбат 3_СТ_МСК,2020,1,55440.0
2,c27d8f86-50b9-49be-9abe-7a9b8ebc8fe3,Архив Метрополис 1_ТЦ_МСК,2018,8,86980.0
3,bc5d3170-e307-468a-9437-0609f3bcf91d,Павелецкий_ТХ_МСК,2018,8,92000.0
4,681f37cd-1166-4ead-a7b9-6c29fd4237b4,Свиблово_ТРЦ_МСК,2019,9,51000.0
...,...,...,...,...,...
93588,e17e4cfd-1022-4815-b534-e5a438a619cb,Невский пр 11_СТ_СПБ,2020,1,57660.0
93589,7ff0857b-c277-4c97-9afb-1575bf5310f1,(АРХИВ)Боткинская 3_ТЦ_СПБ,2018,12,18340.0
93590,48011046-f116-4986-92ac-968f6b2c9d7d,Покровка 14_СТ_МСК,2018,6,63260.0
93591,3888dd66-6e51-4e29-9a9b-136e7c4d265a,Пятницкая 31/2_СТ_МСК,2018,11,58840.0


In [6]:
sales = sales[['Store.Id', 'name','year', 'month', 'sales']].groupby(by=['Store.Id', 'name','year', 'month'], as_index=False).sum()

In [7]:
sales[['year', 'month']].drop_duplicates().sort_values(by = ['year','month'])

,year,month
64,2018,1
65,2018,2
66,2018,3
67,2018,4
68,2018,5
69,2018,6
70,2018,7
71,2018,8
72,2018,9
73,2018,10


In [8]:
sales = sales[(sales.year <= 2019) | ((sales.year == 2020) & (sales.month <= 3))]

In [9]:
sales[['year', 'month']].drop_duplicates().sort_values(by = ['year','month'])

,year,month
64,2018,1
65,2018,2
66,2018,3
67,2018,4
68,2018,5
69,2018,6
70,2018,7
71,2018,8
72,2018,9
73,2018,10


In [10]:
opened_1q_2018 = sales[(sales.year == 2018) & (sales.month <= 3)].name.unique()
print(opened_1q_2018, len(opened_1q_2018))

['Невский пр 81_СТ_СПБ' 'Кузнецкий Мост 12_СТ_МСК' 'Вегас Каширка_ТЦ_МСК'
 'Ривьера_ТЦ_МСК' 'ЦДМ_ТЦ_МСК' 'Архив Арбат' '(Архив)Кофейня Садовая 19'
 'Перловский_ТЦ_МСК' 'Архив Золотой Вавилон Ростокино' 'Архив Тверской'
 'Архив Багратионовская' 'Архив  Сокольники' 'Пятницкая 31/2_СТ_МСК'
 'Шереметьево_ТХ_МО' 'Тверская Застава пл 3_СТ_МСК' 'Покровка 14_СТ_МСК'
 'Архив Люсиновская' '(Архив)Золотой Вавилон Отрадное_ТЦ_МСК'
 'Архив Ленинградский' 'Советская 1Б_СТ_МО' 'Архив(01.09)Мега Химки_ТЦ_МО'
 'Архив Гагаринский_ТЦ_МСК' 'Одоевского 31_СТ_СПБ' 'Архив Водный'
 'Архив Сусальный Н 5_СТ_МСК' 'Авиапарк 3_ТЦ_МСК' 'Архив ТЦ Щука'
 'Архив мякинино' 'Красный Кит_ТЦ_МО' 'Солянка 2/6_СТ_МСК'
 '(Архив 28.06)Город Лефортово_ТЦ_МСК' 'Новомосковский_ТЦ_МО'
 'Охотный Ряд_1_ТЦ_МСК' 'Колумбус 1_ТЦ_МСК' 'Зеленопарк_ТЦ_МО'
 'Мозаика_ТЦ_МСК' 'Инвентарь Одоевского 31'
 'Архив(01.07)Калейдоскоп_ТЦ_МСК' 'Павелецкий_ТХ_МСК'
 'Архив Метрополис 1_ТЦ_МСК' 'Архив Бауманская' 'Ярославский_ТХ_МСК'
 '(Архив) Радуга_ТЦ

In [11]:
working_24m = sales[['name','sales']].groupby(by=['name'], as_index=False).count()
working_24m = list(working_24m[working_24m['sales'] >= 24].name)
print(working_24m, len(working_24m))

['Авиапарк 3_ТЦ_МСК', 'Афимолл 1_ТЦ_МСК', 'Вегас Каширка_ТЦ_МСК', 'Зеленопарк_ТЦ_МО', 'Колумбус 1_ТЦ_МСК', 'Красный Кит_ТЦ_МО', 'Ленинградский 60_СТ_МСК', 'Мозаика_ТЦ_МСК', 'Невский пр 11_СТ_СПБ', 'Невский пр 81_СТ_СПБ', 'Новомосковский_ТЦ_МО', 'Одоевского 31_СТ_СПБ', 'Охотный Ряд_1_ТЦ_МСК', 'Павелецкий_ТХ_МСК', 'Покровка 14_СТ_МСК', 'Пятницкая 31/2_СТ_МСК', 'Ривьера_ТЦ_МСК', 'Солянка 2/6_СТ_МСК', 'ЦДМ_ТЦ_МСК', 'Шереметьево_ТХ_МО', 'Штаер_ТЦ_МСК', 'Ярославский_ТХ_МСК'] 22


In [12]:
stores = sales[sales.name.isin(working_24m)]
stores = stores[['Store.Id', 'name']].drop_duplicates().reset_index(drop=True)
stores

,Store.Id,name
0,0685fda4-5433-48fd-922c-960435e3df6f,Невский пр 81_СТ_СПБ
1,08858cdb-937a-44fe-8e53-a085eb6a4300,Вегас Каширка_ТЦ_МСК
2,1410b4ba-56b1-4ba9-9d58-09287861309f,Ривьера_ТЦ_МСК
3,14920f96-a7cc-4cb4-a5ea-59cba11d6431,ЦДМ_ТЦ_МСК
4,3888dd66-6e51-4e29-9a9b-136e7c4d265a,Пятницкая 31/2_СТ_МСК
5,411abeab-5311-4798-ba63-dd01f00495e1,Шереметьево_ТХ_МО
6,48011046-f116-4986-92ac-968f6b2c9d7d,Покровка 14_СТ_МСК
7,6c950206-0097-48a0-8b8b-a9623ed533db,Одоевского 31_СТ_СПБ
8,780656c4-864e-494b-b1ff-b17426c8f0d3,Авиапарк 3_ТЦ_МСК
9,7deddc00-8a0f-402a-8cb1-6a82b0166da3,Красный Кит_ТЦ_МО


In [13]:
%%time
stores_iiko = pd.read_sql('SELECT * FROM dbo.stores', engine)
stores_iiko = stores_iiko[['open_date', 'format_location_eng', 'iiko_store_id']]
stores_iiko = stores_iiko[stores_iiko.iiko_store_id.isin(stores['Store.Id'])].sort_values(by='open_date')
stores_iiko

Wall time: 1.58 s


,open_date,format_location_eng,iiko_store_id
231,2017-02-14,TC,9c8f2425-8eb4-4263-9b54-26ce18b5512d
274,2017-05-19,Hub,bc5d3170-e307-468a-9437-0609f3bcf91d
179,2017-05-26,TC,780656c4-864e-494b-b1ff-b17426c8f0d3
114,2017-06-29,Hub,411abeab-5311-4798-ba63-dd01f00495e1
191,2017-07-22,TC,7deddc00-8a0f-402a-8cb1-6a82b0166da3
16,2017-07-24,TC,08858cdb-937a-44fe-8e53-a085eb6a4300
301,2017-08-09,TC,cbaf899b-c650-435f-997d-e206cf1942ba
197,2017-08-17,Street,806af309-cf58-4f35-a656-39af5f48e1f0
10,2017-08-30,Street,0685fda4-5433-48fd-922c-960435e3df6f
347,2017-10-07,Street,e17e4cfd-1022-4815-b534-e5a438a619cb


In [14]:
stores = stores.merge(
    stores_iiko, left_on='Store.Id', right_on='iiko_store_id')
stores = stores[['name', 'open_date', 'format_location_eng']].sort_values(by='open_date')
stores.open_date = pd.to_datetime(stores.open_date, format='%Y-%m-%d')
stores['open_year'] = stores.open_date.dt.year
stores['open_month'] = stores.open_date.dt.month
stores

,name,open_date,format_location_eng,open_year,open_month
12,Охотный Ряд_1_ТЦ_МСК,2017-02-14,TC,2017,2
17,Павелецкий_ТХ_МСК,2017-05-19,Hub,2017,5
8,Авиапарк 3_ТЦ_МСК,2017-05-26,TC,2017,5
5,Шереметьево_ТХ_МО,2017-06-29,Hub,2017,6
9,Красный Кит_ТЦ_МО,2017-07-22,TC,2017,7
1,Вегас Каширка_ТЦ_МСК,2017-07-24,TC,2017,7
19,Афимолл 1_ТЦ_МСК,2017-08-09,TC,2017,8
10,Солянка 2/6_СТ_МСК,2017-08-17,Street,2017,8
0,Невский пр 81_СТ_СПБ,2017-08-30,Street,2017,8
21,Невский пр 11_СТ_СПБ,2017-10-07,Street,2017,10


In [15]:
sales = sales.merge(stores, on='name')
sales

,Store.Id,name,year,month,sales,open_date,format_location_eng,open_year,open_month
0,0685fda4-5433-48fd-922c-960435e3df6f,Невский пр 81_СТ_СПБ,2018,1,2387700.000,2017-08-30,Street,2017,8
1,0685fda4-5433-48fd-922c-960435e3df6f,Невский пр 81_СТ_СПБ,2018,2,2186800.000,2017-08-30,Street,2017,8
2,0685fda4-5433-48fd-922c-960435e3df6f,Невский пр 81_СТ_СПБ,2018,3,2798780.000,2017-08-30,Street,2017,8
3,0685fda4-5433-48fd-922c-960435e3df6f,Невский пр 81_СТ_СПБ,2018,4,3270160.000,2017-08-30,Street,2017,8
4,0685fda4-5433-48fd-922c-960435e3df6f,Невский пр 81_СТ_СПБ,2018,5,3545160.000,2017-08-30,Street,2017,8
...,...,...,...,...,...,...,...,...,...
575,e17e4cfd-1022-4815-b534-e5a438a619cb,Невский пр 11_СТ_СПБ,2019,11,1858460.000,2017-10-07,Street,2017,10
576,e17e4cfd-1022-4815-b534-e5a438a619cb,Невский пр 11_СТ_СПБ,2019,12,1811740.000,2017-10-07,Street,2017,10
577,e17e4cfd-1022-4815-b534-e5a438a619cb,Невский пр 11_СТ_СПБ,2020,1,1844640.000,2017-10-07,Street,2017,10
578,e17e4cfd-1022-4815-b534-e5a438a619cb,Невский пр 11_СТ_СПБ,2020,2,1614329.988,2017-10-07,Street,2017,10


In [18]:
sales = sales.iloc[:,1:]
sales

,name,year,month,sales,open_date,format_location_eng,open_year,open_month
0,Невский пр 81_СТ_СПБ,2018,1,2387700.000,2017-08-30,Street,2017,8
1,Невский пр 81_СТ_СПБ,2018,2,2186800.000,2017-08-30,Street,2017,8
2,Невский пр 81_СТ_СПБ,2018,3,2798780.000,2017-08-30,Street,2017,8
3,Невский пр 81_СТ_СПБ,2018,4,3270160.000,2017-08-30,Street,2017,8
4,Невский пр 81_СТ_СПБ,2018,5,3545160.000,2017-08-30,Street,2017,8
...,...,...,...,...,...,...,...,...
575,Невский пр 11_СТ_СПБ,2019,11,1858460.000,2017-10-07,Street,2017,10
576,Невский пр 11_СТ_СПБ,2019,12,1811740.000,2017-10-07,Street,2017,10
577,Невский пр 11_СТ_СПБ,2020,1,1844640.000,2017-10-07,Street,2017,10
578,Невский пр 11_СТ_СПБ,2020,2,1614329.988,2017-10-07,Street,2017,10


In [30]:
# Добавим в расчёт колонку с месяцем работы кофейни
sales = sales.copy()
sales['age'] = sales.apply(lambda x: x.month - x.open_month + 12*(x.year-x.open_year) + 1, axis=1)
sales

,name,year,month,sales,open_date,format_location_eng,open_year,open_month,age
0,Невский пр 81_СТ_СПБ,2018,1,2387700.000,2017-08-30,Street,2017,8,6
1,Невский пр 81_СТ_СПБ,2018,2,2186800.000,2017-08-30,Street,2017,8,7
2,Невский пр 81_СТ_СПБ,2018,3,2798780.000,2017-08-30,Street,2017,8,8
3,Невский пр 81_СТ_СПБ,2018,4,3270160.000,2017-08-30,Street,2017,8,9
4,Невский пр 81_СТ_СПБ,2018,5,3545160.000,2017-08-30,Street,2017,8,10
...,...,...,...,...,...,...,...,...,...
575,Невский пр 11_СТ_СПБ,2019,11,1858460.000,2017-10-07,Street,2017,10,26
576,Невский пр 11_СТ_СПБ,2019,12,1811740.000,2017-10-07,Street,2017,10,27
577,Невский пр 11_СТ_СПБ,2020,1,1844640.000,2017-10-07,Street,2017,10,28
578,Невский пр 11_СТ_СПБ,2020,2,1614329.988,2017-10-07,Street,2017,10,29


In [172]:
pivot = pd.pivot_table(sales, index=['name', 'open_date', 'format_location_eng'], columns='age',
                       values='sales').sort_values(by='open_date').reset_index()
display(pivot)

age,name,open_date,format_location_eng,1,2,3,4,5,6,7,...,29,30,31,32,33,34,35,36,37,38
0,Охотный Ряд_1_ТЦ_МСК,2017-02-14,TC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3300279.960,3595579.97,3360340.00,3891140.000,4300640.000,4025440.0,4724900.00,3738540.0,3429420.0,2330440.002
1,Павелецкий_ТХ_МСК,2017-05-19,Hub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2703179.980,2808880.00,2665215.99,2962659.990,1922460.010,38160.0,1711209.96,NaN,NaN,NaN
2,Авиапарк 3_ТЦ_МСК,2017-05-26,TC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3302920.000,3621300.00,3773340.00,3876740.000,3612540.000,3305880.0,2095260.00,NaN,NaN,NaN
3,Шереметьево_ТХ_МО,2017-06-29,Hub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2285960.000,1963120.00,2070220.00,1742520.000,1551660.000,1167499.8,NaN,NaN,NaN,NaN
4,Красный Кит_ТЦ_МО,2017-07-22,TC,NaN,NaN,NaN,NaN,NaN,NaN,1517920.00,...,1698920.000,1714300.00,1451279.91,1359479.952,1137960.000,NaN,NaN,NaN,NaN,NaN
5,Вегас Каширка_ТЦ_МСК,2017-07-24,TC,NaN,NaN,NaN,NaN,NaN,NaN,1657310.00,...,2209248.000,2353580.00,1993668.00,1780716.006,1373808.042,NaN,NaN,NaN,NaN,NaN
6,Афимолл 1_ТЦ_МСК,2017-08-09,TC,NaN,NaN,NaN,NaN,NaN,2007520.00,2296180.00,...,2921920.000,2159520.00,2265540.00,1619899.992,NaN,NaN,NaN,NaN,NaN,NaN
7,Солянка 2/6_СТ_МСК,2017-08-17,Street,NaN,NaN,NaN,NaN,NaN,1639010.00,1678000.00,...,2727100.000,2244839.97,2444279.99,1839639.984,NaN,NaN,NaN,NaN,NaN,NaN
8,Невский пр 81_СТ_СПБ,2017-08-30,Street,NaN,NaN,NaN,NaN,NaN,2387700.00,2186800.00,...,3143106.990,2911680.00,2632079.88,2218289.880,NaN,NaN,NaN,NaN,NaN,NaN
9,Невский пр 11_СТ_СПБ,2017-10-07,Street,NaN,NaN,NaN,1385460.00,1197300.00,1533600.00,2129260.00,...,1614329.988,1328579.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
sales[sales.open_date < '2018-01-01']

,name,year,month,sales,open_date,format_location_eng,open_year,open_month,age
0,Невский пр 81_СТ_СПБ,2018,1,2387700.000,2017-08-30,Street,2017,8,6
1,Невский пр 81_СТ_СПБ,2018,2,2186800.000,2017-08-30,Street,2017,8,7
2,Невский пр 81_СТ_СПБ,2018,3,2798780.000,2017-08-30,Street,2017,8,8
3,Невский пр 81_СТ_СПБ,2018,4,3270160.000,2017-08-30,Street,2017,8,9
4,Невский пр 81_СТ_СПБ,2018,5,3545160.000,2017-08-30,Street,2017,8,10
...,...,...,...,...,...,...,...,...,...
575,Невский пр 11_СТ_СПБ,2019,11,1858460.000,2017-10-07,Street,2017,10,26
576,Невский пр 11_СТ_СПБ,2019,12,1811740.000,2017-10-07,Street,2017,10,27
577,Невский пр 11_СТ_СПБ,2020,1,1844640.000,2017-10-07,Street,2017,10,28
578,Невский пр 11_СТ_СПБ,2020,2,1614329.988,2017-10-07,Street,2017,10,29


In [174]:
pivot

age,name,open_date,format_location_eng,1,2,3,4,5,6,7,...,29,30,31,32,33,34,35,36,37,38
0,Охотный Ряд_1_ТЦ_МСК,2017-02-14,TC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3300279.960,3595579.97,3360340.00,3891140.000,4300640.000,4025440.0,4724900.00,3738540.0,3429420.0,2330440.002
1,Павелецкий_ТХ_МСК,2017-05-19,Hub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2703179.980,2808880.00,2665215.99,2962659.990,1922460.010,38160.0,1711209.96,NaN,NaN,NaN
2,Авиапарк 3_ТЦ_МСК,2017-05-26,TC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3302920.000,3621300.00,3773340.00,3876740.000,3612540.000,3305880.0,2095260.00,NaN,NaN,NaN
3,Шереметьево_ТХ_МО,2017-06-29,Hub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2285960.000,1963120.00,2070220.00,1742520.000,1551660.000,1167499.8,NaN,NaN,NaN,NaN
4,Красный Кит_ТЦ_МО,2017-07-22,TC,NaN,NaN,NaN,NaN,NaN,NaN,1517920.00,...,1698920.000,1714300.00,1451279.91,1359479.952,1137960.000,NaN,NaN,NaN,NaN,NaN
5,Вегас Каширка_ТЦ_МСК,2017-07-24,TC,NaN,NaN,NaN,NaN,NaN,NaN,1657310.00,...,2209248.000,2353580.00,1993668.00,1780716.006,1373808.042,NaN,NaN,NaN,NaN,NaN
6,Афимолл 1_ТЦ_МСК,2017-08-09,TC,NaN,NaN,NaN,NaN,NaN,2007520.00,2296180.00,...,2921920.000,2159520.00,2265540.00,1619899.992,NaN,NaN,NaN,NaN,NaN,NaN
7,Солянка 2/6_СТ_МСК,2017-08-17,Street,NaN,NaN,NaN,NaN,NaN,1639010.00,1678000.00,...,2727100.000,2244839.97,2444279.99,1839639.984,NaN,NaN,NaN,NaN,NaN,NaN
8,Невский пр 81_СТ_СПБ,2017-08-30,Street,NaN,NaN,NaN,NaN,NaN,2387700.00,2186800.00,...,3143106.990,2911680.00,2632079.88,2218289.880,NaN,NaN,NaN,NaN,NaN,NaN
9,Невский пр 11_СТ_СПБ,2017-10-07,Street,NaN,NaN,NaN,1385460.00,1197300.00,1533600.00,2129260.00,...,1614329.988,1328579.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
pivot.to_excel('G:/Мой диск/01 Нерегулярные отчёты/201001 Rump Up/data_from_dwh.xlsx', index=False)